TODO: 

* setup streams using python snowflake connector

In [1]:
import snowflake.connector
import os

/Users/jordanpacho/Desktop/Sigma_Labs/Coursework/Deloton-Exercise-Bikes/.venv/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (9.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
USER = os.environ.get('USER')
ACCOUNT = os.environ.get('ACCOUNT')
PASSWORD = os.environ.get('PASSWORD')
WAREHOUSE= os.environ.get('WAREHOUSE')
DATABASE= os.environ.get('DATABASE')
BATCH_SCHEMA= os.environ.get('BATCH_SCHEMA')

In [3]:
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    warehouse=WAREHOUSE,
    database=DATABASE,
    schema=BATCH_SCHEMA
)
cs = conn.cursor()

#### Making table streams

In [4]:
query = """
    CREATE STREAM rides_stream
    ON TABLE rides
"""
# cs.execute(query)

In [5]:
query_stream = """
    SELECT *
    FROM rides_stream
"""

cs.execute(query_stream).fetch_pandas_all()

,RIDE_ID,USER_ID,BEGIN_TIMESTAMP,TOTAL_DURATION_SEC,TOTAL_POWER,MEAN_POWER,MEAN_RESISTANCE,MEAN_RPM,MEAN_HEART_RATE,METADATA$ACTION,METADATA$ISUPDATE,METADATA$ROW_ID
0,52,4610,"{'epoch': 1665063164, 'fraction': 0}",520,29818.325,57.342933,42.107692,47.300000,111.944231,INSERT,False,b7ec5aaa911a775b92a3e4901ce5be7f351f569c
1,48,4608,"{'epoch': 1665057717, 'fraction': 0}",490,22375.036,45.663339,40.579592,49.783673,107.720408,INSERT,False,0c4a6f1ad74772ba94a4aac6b5b6819c52e947bf
2,51,4609,"{'epoch': 1665059254, 'fraction': 0}",520,20755.968,39.915323,40.953846,43.659615,91.730769,INSERT,False,d00a0033a02e3252dca09a6807e688c5eccdf7c9
3,53,4610,"{'epoch': 1665063687, 'fraction': 0}",520,30266.589,58.204979,41.938462,46.584615,136.876923,INSERT,False,398f99b5d8f8b8424335b3569368babc7fe0b175
4,47,4608,"{'epoch': 1665057197, 'fraction': 0}",517,20884.968,40.396456,40.015474,41.827853,114.294004,INSERT,False,17d703d74565e9c741bce3bf0b8b46e0fc630060
5,49,4608,"{'epoch': 1665058209, 'fraction': 0}",520,18181.857,34.965110,39.900000,52.311538,104.361538,INSERT,False,2fe1e68fe1479ada14fa27202a9632e84901d6cd
6,50,4609,"{'epoch': 1665058732, 'fraction': 0}",519,24850.436,47.881380,40.897881,37.104046,111.263969,INSERT,False,84caf69c94c0a32bc51c3f0ec78a2074bf5c4811


In [6]:
query = """
    SELECT user_id
        FROM users
"""

cs.execute(query).fetch_pandas_all()

,USER_ID
0,4603
1,4604
2,4607
3,4608
4,4609
5,4610


In [7]:
query = """
    SELECT user_id
        FROM rides_stream
"""

cs.execute(query).fetch_pandas_all()

,USER_ID
0,4608
1,4609
2,4608
3,4610
4,4609
5,4608
6,4610


In [8]:
query = """
    SELECT users.user_id, rides_stream.ride_id
        FROM users
        RIGHT JOIN rides_stream
        ON users.user_id = rides_stream.user_id
"""

cs.execute(query).fetch_pandas_all()

,USER_ID,RIDE_ID
0,4608,47
1,4608,48
2,4608,49
3,4609,51
4,4609,50
5,4610,53
6,4610,52


In [ ]:
create or replace TABLE DELATON.ZOOKEEPERS_BATCH_PRODUCTION.USERS (
	USER_ID NUMBER(38,0) NOT NULL,
	FIRST_NAME VARCHAR(50),
	LAST_NAME VARCHAR(50),
	GENDER VARCHAR(6),
	DATE_OF_BIRTH DATE,
	HEIGHT_CM NUMBER(38,0),
	WEIGHT_KG NUMBER(38,0),
	HOUSE_NAME VARCHAR(25),
	STREET VARCHAR(50),
	REGION VARCHAR(50),
	POSTCODE VARCHAR(8),
	EMAIL VARCHAR(50),
	ACCOUNT_CREATED DATE,
	primary key (USER_ID)
);

In [ ]:
create or replace TABLE DELATON.ZOOKEEPERS_BATCH_PRODUCTION.RIDES (
	RIDE_ID NUMBER(38,0) NOT NULL DEFAULT DELATON.ZOOKEEPERS_BATCH_PRODUCTION.SEQ1.NEXTVAL,
	USER_ID NUMBER(38,0),
	BEGIN_TIMESTAMP TIMESTAMP_NTZ(9) NOT NULL,
	TOTAL_DURATION_SEC NUMBER(38,0) NOT NULL,
	TOTAL_POWER FLOAT NOT NULL,
	MEAN_POWER FLOAT NOT NULL,
	MEAN_RESISTANCE FLOAT NOT NULL,
	MEAN_RPM FLOAT NOT NULL,
	MEAN_HEART_RATE FLOAT NOT NULL,
	primary key (RIDE_ID)
);

In [24]:
query = """
    CREATE OR REPLACE TABLE DELATON.ZOOKEEPERS_BATCH_PRODUCTION.RECENT_RIDES (
        USER_ID NUMBER(38,0) NOT NULL,
        RIDE_ID NUMBER(38,0) NOT NULL,
        GENDER VARCHAR(6) NOT NULL,
        AGE NUMBER(38,0) NOT NULL,
        BEGIN_TIMESTAMP TIMESTAMP_NTZ(9) NOT NULL,
        TOTAL_DURATION_SEC NUMBER(38,0) NOT NULL,
        TOTAL_POWER FLOAT NOT NULL,
        MEAN_POWER FLOAT NOT NULL,
        MEAN_RESISTANCE FLOAT NOT NULL,
        MEAN_RPM FLOAT NOT NULL,
        MEAN_HEART_RATE FLOAT NOT NULL
    )
"""

cs.execute(query)

In [37]:
query = """
    INSERT INTO recent_rides
        WITH recent_rides_data AS(

            WITH user_gender_dob AS (
            SELECT user_id, gender,
                DATEDIFF(hour,date_of_birth,GETDATE())/8766000000 AS age
                FROM users
            )
            
            SELECT ugd.user_id, rs.ride_id, ugd.gender, ugd.age, rs.begin_timestamp, rs.total_duration_sec, rs.total_power,
                rs.mean_power, rs.mean_resistance, rs.mean_rpm, rs.mean_heart_rate

                FROM user_gender_dob AS ugd
                RIGHT JOIN rides_stream AS rs
                    ON ugd.user_id = rs.user_id
                WHERE TO_DATE(rs.begin_timestamp) > DATEADD(HOUR, -12, GETDATE())
        )
        SELECT * FROM recent_rides_data
     
"""

cs.execute(query)

In [38]:
query = """
   SELECT * 
    FROM recent_rides
"""

cs.execute(query).fetch_pandas_all()

,USER_ID,RIDE_ID,GENDER,AGE,BEGIN_TIMESTAMP,TOTAL_DURATION_SEC,TOTAL_POWER,MEAN_POWER,MEAN_RESISTANCE,MEAN_RPM,MEAN_HEART_RATE
0,4612,55,Female,0,"{'epoch': 1665067185, 'fraction': 0}",520,33838.639,65.074306,42.792308,44.376923,90.353846
1,4608,48,Female,0,"{'epoch': 1665057717, 'fraction': 0}",490,22375.036,45.663339,40.579592,49.783673,107.720408
2,4608,49,Female,0,"{'epoch': 1665058209, 'fraction': 0}",520,18181.857,34.965110,39.900000,52.311538,104.361538
3,4608,47,Female,0,"{'epoch': 1665057197, 'fraction': 0}",517,20884.968,40.396456,40.015474,41.827853,114.294004
4,4609,50,Male,0,"{'epoch': 1665058732, 'fraction': 0}",519,24850.436,47.881380,40.897881,37.104046,111.263969
5,4609,51,Male,0,"{'epoch': 1665059254, 'fraction': 0}",520,20755.968,39.915323,40.953846,43.659615,91.730769
6,4611,54,Female,0,"{'epoch': 1665066662, 'fraction': 0}",520,16769.120,32.248308,37.353846,42.351923,124.136538
7,4610,53,Male,0,"{'epoch': 1665063687, 'fraction': 0}",520,30266.589,58.204979,41.938462,46.584615,136.876923
8,4610,52,Male,0,"{'epoch': 1665063164, 'fraction': 0}",520,29818.325,57.342933,42.107692,47.300000,111.944231
